In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 22683.16it/s]


In [3]:
len(human_vocab)

37

In [4]:
tx = 30
ty = 10
x, y, xoh, yoh = preprocess_data(dataset, human_vocab, machine_vocab, tx, ty)
'''
Xoh: one-hot version of X, the "1" entry's index is mapped to the character thanks to human_vocab. Xoh.shape = (m, Tx, len(human_vocab))
Yoh: one-hot version of Y, the "1" entry's index is mapped to the character thanks to machine_vocab. Yoh.shape = (m, Tx, len(machine_vocab)). Here, len(machine_vocab) = 11 since there are 11 characters ('-' as well as 0-9).
'''

'\nXoh: one-hot version of X, the "1" entry\'s index is mapped to the character thanks to human_vocab. Xoh.shape = (m, Tx, len(human_vocab))\nYoh: one-hot version of Y, the "1" entry\'s index is mapped to the character thanks to machine_vocab. Yoh.shape = (m, Tx, len(machine_vocab)). Here, len(machine_vocab) = 11 since there are 11 characters (\'-\' as well as 0-9).\n'

In [5]:
repeator = RepeatVector(tx)
concatenator  = Concatenate(axis = -1)
densor1 = Dense(10, activation = 'tanh')
densor2 = Dense(1, activation = 'relu')
activator = Activation(softmax, name = 'attention_weights')
dotor = Dot(axes= 1)

In [6]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [7]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state=True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [8]:
def model(tx, ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape = (tx, human_vocab_size))
    s0 = Input(shape=(n_s, ), name='s0')
    c0 = Input(shape=(n_s, ), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    for t in range(ty):
        context = one_step_attention(a, s)
        
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        out = output_layer(s)
        
        outputs.append(out)
        
    model = Model(inputs = [X, s0, c0], outputs = outputs)
    
    return model

In [9]:
model = model(tx,ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [10]:
model.summary() 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

                                                                 attention_weights[4][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[5][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[6][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[7][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[8][0]          
                                                                 bidirectional_1[0][0]            
          

In [11]:
opt = Adam(lr = 0.005, beta_1 = 0.9, beta_2 = 0.999,decay = 0.01)  
model.compile(loss = 'categorical_crossentropy', optimizer = opt,metrics = ['accuracy']) 

In [12]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))

In [14]:
model.fit([xoh, s0, c0], outputs, epochs=1, batch_size=100)

Epoch 1/1


 2800/10000 [=======>......................] - ETA: 9:40 - loss: 23.9797 - dense_3_loss: 2.3984 - dense_3_acc: 0.0100 - dense_3_acc_1: 0.4200 - dense_3_acc_2: 0.1300 - dense_3_acc_3: 0.1000 - dense_3_acc_4: 0.1100 - dense_3_acc_5: 0.0200 - dense_3_acc_6: 0.0600 - dense_3_acc_7: 0.0600 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.090 - ETA: 4:51 - loss: 23.8178 - dense_3_loss: 2.4017 - dense_3_acc: 0.0050 - dense_3_acc_1: 0.2100 - dense_3_acc_2: 0.0650 - dense_3_acc_3: 0.0500 - dense_3_acc_4: 0.5500 - dense_3_acc_5: 0.0150 - dense_3_acc_6: 0.0300 - dense_3_acc_7: 0.5200 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.045 - ETA: 3:14 - loss: 23.6374 - dense_3_loss: 2.4176 - dense_3_acc: 0.0033 - dense_3_acc_1: 0.1400 - dense_3_acc_2: 0.0433 - dense_3_acc_3: 0.0333 - dense_3_acc_4: 0.7000 - dense_3_acc_5: 0.0100 - dense_3_acc_6: 0.0200 - dense_3_acc_7: 0.6800 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.030 - ETA: 2:26 - loss: 23.4099 - dense_3_loss: 2.4359 - dense_3_acc: 0.0025 - dens

 5700/10000 [================>.............] - ETA: 19s - loss: 21.2402 - dense_3_loss: 2.8051 - dense_3_acc: 3.4483e-04 - dense_3_acc_1: 0.3814 - dense_3_acc_2: 0.1745 - dense_3_acc_3: 0.0624 - dense_3_acc_4: 0.8572 - dense_3_acc_5: 0.0514 - dense_3_acc_6: 0.0155 - dense_3_acc_7: 0.8648 - dense_3_acc_8: 0.0290 - dense_3_acc_9: 0.01 - ETA: 18s - loss: 21.1933 - dense_3_loss: 2.8068 - dense_3_acc: 3.3333e-04 - dense_3_acc_1: 0.3893 - dense_3_acc_2: 0.1777 - dense_3_acc_3: 0.0603 - dense_3_acc_4: 0.8620 - dense_3_acc_5: 0.0497 - dense_3_acc_6: 0.0150 - dense_3_acc_7: 0.8693 - dense_3_acc_8: 0.0280 - dense_3_acc_9: 0.01 - ETA: 18s - loss: 21.1387 - dense_3_loss: 2.8019 - dense_3_acc: 3.2258e-04 - dense_3_acc_1: 0.3961 - dense_3_acc_2: 0.1794 - dense_3_acc_3: 0.0584 - dense_3_acc_4: 0.8665 - dense_3_acc_5: 0.0481 - dense_3_acc_6: 0.0145 - dense_3_acc_7: 0.8735 - dense_3_acc_8: 0.0271 - dense_3_acc_9: 0.01 - ETA: 17s - loss: 21.0859 - dense_3_loss: 2.8027 - dense_3_acc: 3.1250e-04 - dense_3

 8600/10000 [========================>.....] - ETA: 7s - loss: 19.7691 - dense_3_loss: 2.7760 - dense_3_acc: 0.2366 - dense_3_acc_1: 0.5057 - dense_3_acc_2: 0.2381 - dense_3_acc_3: 0.0559 - dense_3_acc_4: 0.9286 - dense_3_acc_5: 0.0697 - dense_3_acc_6: 0.0095 - dense_3_acc_7: 0.8467 - dense_3_acc_8: 0.1190 - dense_3_acc_9: 0.060 - ETA: 7s - loss: 19.7067 - dense_3_loss: 2.7724 - dense_3_acc: 0.2434 - dense_3_acc_1: 0.5081 - dense_3_acc_2: 0.2400 - dense_3_acc_3: 0.0569 - dense_3_acc_4: 0.9298 - dense_3_acc_5: 0.0771 - dense_3_acc_6: 0.0105 - dense_3_acc_7: 0.8490 - dense_3_acc_8: 0.1175 - dense_3_acc_9: 0.062 - ETA: 6s - loss: 19.6451 - dense_3_loss: 2.7695 - dense_3_acc: 0.2505 - dense_3_acc_1: 0.5118 - dense_3_acc_2: 0.2430 - dense_3_acc_3: 0.0560 - dense_3_acc_4: 0.9310 - dense_3_acc_5: 0.0893 - dense_3_acc_6: 0.0105 - dense_3_acc_7: 0.8515 - dense_3_acc_8: 0.1207 - dense_3_acc_9: 0.062 - ETA: 6s - loss: 19.5846 - dense_3_loss: 2.7682 - dense_3_acc: 0.2566 - dense_3_acc_1: 0.5148 - 

10000/10000 [==============================] - ETA: 1s - loss: 17.6046 - dense_3_loss: 2.6494 - dense_3_acc: 0.3675 - dense_3_acc_1: 0.5914 - dense_3_acc_2: 0.2878 - dense_3_acc_3: 0.0718 - dense_3_acc_4: 0.9524 - dense_3_acc_5: 0.3000 - dense_3_acc_6: 0.0344 - dense_3_acc_7: 0.8976 - dense_3_acc_8: 0.2054 - dense_3_acc_9: 0.087 - ETA: 1s - loss: 17.5361 - dense_3_loss: 2.6459 - dense_3_acc: 0.3709 - dense_3_acc_1: 0.5940 - dense_3_acc_2: 0.2885 - dense_3_acc_3: 0.0719 - dense_3_acc_4: 0.9530 - dense_3_acc_5: 0.3050 - dense_3_acc_6: 0.0364 - dense_3_acc_7: 0.8987 - dense_3_acc_8: 0.2073 - dense_3_acc_9: 0.087 - ETA: 1s - loss: 17.4681 - dense_3_loss: 2.6423 - dense_3_acc: 0.3734 - dense_3_acc_1: 0.5971 - dense_3_acc_2: 0.2901 - dense_3_acc_3: 0.0725 - dense_3_acc_4: 0.9535 - dense_3_acc_5: 0.3092 - dense_3_acc_6: 0.0397 - dense_3_acc_7: 0.8999 - dense_3_acc_8: 0.2087 - dense_3_acc_9: 0.088 - ETA: 1s - loss: 17.4008 - dense_3_loss: 2.6383 - dense_3_acc: 0.3764 - dense_3_acc_1: 0.6009 - 

In [15]:
model.load_weights('models/model.h5')

In [44]:
examples = ['3 may 1979', '5 april 09', '21th of august 2016', 'tue 10 jul 2007', 'saturday may 9 2018', 
            'march 3 2001', 'march 3rd 2001', '1 march 2001']
for example in examples:
    source  = string_to_int(example, tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape(1, 30,37)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis=-1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print('source:', example)
    print('output:', ''.join(output))

source: 3 may 1979
output: 1979-05-03
source: 5 april 09
output: 2009-05-05
source: 21th of august 2016
output: 2016-08-21
source: tue 10 jul 2007
output: 2007-07-10
source: saturday may 9 2018
output: 2018-05-09
source: march 3 2001
output: 2001-03-03
source: march 3rd 2001
output: 2001-03-03
source: 1 march 2001
output: 2001-03-01
